In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os
from pathlib import Path

In [2]:
from PIL import Image
import cv2
# import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [4]:
import json
import pickle
import re
from tqdm import tqdm

### 0.API

In [ ]:
sys.path.append('../../')
from utils.plots.plots import draw_bbox

In [ ]:
# id_standard_df.to_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv', index=False)

In [ ]:
id_standard_df = pd.read_csv('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/id2class99_chenyi.csv')
id_use_df = id_standard_df[id_standard_df['use']>=0]

In [ ]:
with open('/home/chenyi/workspace/myPyModule99/dataset/labelMap/standard/haowei34k_map_stand.json', 'r') as f:
    hw2stand = json.load(f)

In [5]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [6]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [ ]:
# l2_map_l1 = get_list2dict( id_use_df['label99'].values, id_use_df['label_l1'].values)

In [7]:
label2model = get_list2dict( model_label_map.values(), model_label_map.keys())

### 1.Dataset Information.

In [ ]:
import numpy as np

In [ ]:
# openfashion_pd.to_csv('./data/hw34k_bbox_info.csv', index=False)
openfashion_pd = pd.read_csv('./data/hw34k_bbox_info.csv')

In [ ]:
# p = '/home/chenyi/workspace/myPyModule99/data_format/haowei34k_voc_bboxes_relabel.csv'
# openfashion_pd.to_csv(p, index=False)

In [ ]:
openfashion_pd.reset_index(inplace=True, drop=True)
openfashion_pd['folder'] = openfashion_pd.apply(lambda x: x, axis=1)

In [ ]:
openfashion_pd['label1_chi'] = -1
openfashion_pd['label1_chi'] = openfashion_pd['label8'].map(id2eng_l1)

In [ ]:
openfashion_pd['label2_eng'] = openfashion_pd['label99'].map(id2eng_l2)

In [ ]:
openfashion_pd['img_tp'] = openfashion_pd['img_sp'].apply(lambda x:)

### 画图

In [ ]:
df = openfashion_pd

In [ ]:
df.head(2)

In [ ]:
gp = dict(list(df.groupby('img_sp')))

In [ ]:
gp[list(gp.keys())[0]]

In [ ]:
import cv2
TROOT = '/nas/chenyi/datasets_cls/haowei34k_drawed/images/'
for k,v in tqdm(gp.items()):
    k_p = Path(k)
    img_np = cv2.imread(k)
    # img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    bboxes = v.loc[:, ['label2_eng', 'xmin', 'ymin', 'xmax', 'ymax']].values
    mode = v['mode'].values[0]
    tp = Path('{0}{1}/{2}/{3}'.format(TROOT, mode, np.random.randint(0,1000), k_p.name))
    for bx in bboxes:
        img_np = draw_bbox(img_np, bx, fontScale=1)
    if not tp.parent.is_dir():
        tp.parent.mkdir(exist_ok=True, parents=True)
    try:
        cv2.imwrite(str(tp),img_np)
    except:
        print('error: p')
    # break

In [ ]:
Image.fromarray(a)

In [ ]:
slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

In [ ]:
slink_df.head(2)

In [ ]:
fname2path = get_list2dict(slink_df['fname'].values, slink_df['image_spath'].values)

In [ ]:
openfashion_pd['img_sp'] = openfashion_pd['fname'].map(fname2path)

In [ ]:
openfashion_pd.head()

In [ ]:
counter = openfashion_pd.value_counts('label8').sort_index()

In [ ]:
id_list = np.array(counter.axes)[0]
numbers = counter.values
name_list = [str(x) + ':' + id2eng_l1[x] for x in id_list]

In [ ]:
plt.figure(figsize=(25, 7))
plt.xticks(list(range(len(name_list))), name_list, rotation=75, fontsize=14)
plt.bar(name_list, numbers, width=0.5)  # 绘制y刻度标签
for a, b in zip(name_list, numbers):
    plt.text(a, b, b, ha='center', va='bottom')
plt.show()

### target label

In [8]:
!tree -d -L 2 /home/chenyi/workspace/dataset/labels/

/home/chenyi/workspace/dataset/labels/
├── dada_det
│   ├── labels100k_skirt_df2
│   ├── labels202205
│   ├── labels202206
│   └── labels_sys_error
└── labels6
    ├── fashionpedia
    ├── haowei34k
    └── labels_exp

9 directories


In [16]:
# !mkdir -p /home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels

In [17]:
df = pd.read_csv('./data/exps/dadetGallv4_del.csv')

In [18]:
len(df)

219575

In [19]:
df.value_counts('label_model')

label_model
0    84461
2    52819
3    27599
1    22206
5    22190
4    10300
dtype: int64

In [12]:
df = df[df['label_model']>=0]

#### source label path

In [20]:
df.head(1)

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode,stem
0,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,683.0,1024.0,0.461933,0.87793,0.074671,0.136719,0,train,c13d5c79b7bd1071eeb288d809c0ab52


In [21]:
!ls /home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels

In [22]:
root = '/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/'

In [23]:
df['label_tpath'] = df.apply(lambda row: Path('{0}{1}/{2}.txt'.format(root, row['mode'], row['stem'])), axis=1)

In [ ]:
# df.to_csv('./data/hw34k_bbox_info.csv', index=False)

#### 生成txt

In [24]:
sys.path.append('../../')

In [25]:
from utils.bbox.conversation import bbox_coco2voc,bbox_voc2yolo,bbox_yolo2voc

In [27]:
df.head(2)

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode,stem,label_tpath
0,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,683.0,1024.0,0.461933,0.87793,0.074671,0.136719,0,train,c13d5c79b7bd1071eeb288d809c0ab52,/home/chenyi/workspace/dataset/labels/labels6/...
1,haowei34k,/home/chenyi/workspace/dataset/data/haowei34k/...,800.0,1000.0,0.491250,0.92050,0.062500,0.077000,0,train,20170510091134139,/home/chenyi/workspace/dataset/labels/labels6/...


In [ ]:
df['label_model'] = df['label8'].map(label2model)

In [28]:
df.value_counts('label_model')

label_model
0    84461
2    52819
3    27599
1    22206
5    22190
4    10300
dtype: int64

In [29]:
bboxes_list = []
for ind,row in tqdm(df.iterrows()):
    bbox7 = row.loc[['label_model', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
    bbox5 = bbox7[:6]
    # bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
    bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
    bboxes_list.append(bbox5_str)
    # break

219575it [00:45, 4827.55it/s]


In [ ]:
# bboxes_list = []
# for ind,row in tqdm(df.iterrows()):
#     bbox7 = row.loc[['label_model', 'xmin', 'ymin', 'xmax', 'ymax', 'width', 'height']].values.tolist()
#     bbox5 = bbox7[:6]
#     # bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
#     bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
#     bboxes_list.append(bbox5_str)
#     # break

In [26]:
bboxes_list[-1]

'0 0.291781 0.864084 0.113699 0.161387\n'

In [30]:
def write_txt(label_path, text):
    # label_path, text = input_list[0], input_list[1]
    label_path = Path(label_path)
    p_parent = label_path.parent
    if not p_parent.is_dir():
        p_parent.mkdir(parents=True)
    if not p_parent.is_file():
        with open(label_path, 'a+') as f:
            f.write(text)

In [ ]:
# import shutil
# if Path(df['label_tpath'].values[0]).parents[1].is_dir():
#     shutil.rmtree(Path(df['label_tpath'].values[0]).parents[1])

In [31]:
for label_path, text in tqdm(zip(df['label_tpath'].values, bboxes_list)):
    write_txt(label_path, text)
    # break

219575it [00:06, 34537.20it/s]


In [32]:
label_path,text

(PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/val/20140305215504279.txt'),
 '0 0.552885 0.824800 0.091346 0.059200\n')

In [33]:
!cat $label_path

5 0.451923 0.510400 0.302885 0.553600
1 0.638221 0.448000 0.161058 0.150400
0 0.486779 0.851200 0.074519 0.080000
0 0.552885 0.824800 0.091346 0.059200


#### 生成软连接的path

In [36]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/

/home/chenyi/workspace/dataset/data/
├── dada_det
├── data_fashionpedia
├── deepedia
├── DeepFashion2
├── haowei34k
└── tar

6 directories


In [37]:
!tree -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

17 directories


In [36]:
# !mkdir -p /home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox

In [38]:
df.head(1)

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode,stem,label_tpath
0,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,683.0,1024.0,0.461933,0.87793,0.074671,0.136719,0,train,c13d5c79b7bd1071eeb288d809c0ab52,/home/chenyi/workspace/dataset/labels/labels6/...


In [39]:
drop_df = df.drop_duplicates('label_tpath')

In [40]:
slink_df = pd.DataFrame()
slink_df['fname'] = drop_df['stem']
slink_df['label_sp'] = drop_df['label_tpath']
slink_df['img_sp'] = drop_df['img_sp']

In [41]:
# root = Path('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/')
# img_root = Path('/home/chenyi/workspace/dataset//')
yolo_data_root = Path('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/')

In [42]:
slink_df['label_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'labels' /x.parts[-2]/x.parts[-1])

In [43]:
slink_df['img_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'images' /x.parts[-2]/x.parts[-1])
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: x.with_suffix('.jpg'))

In [44]:
# slink_df['img_sp'] = slink_df['img_tp'].apply(lambda x: data_root /'images' /x.parts[-2]/x.parts[-1])

In [46]:
slink_df.to_csv('./data/gall10k_delBox_slink.csv', index=False)

In [47]:
slink_df.head(1).values

array([['c13d5c79b7bd1071eeb288d809c0ab52',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt'),
        '/home/chenyi/workspace/dataset/data/data_fashionpedia/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg',
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt'),
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/images/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg')]],
      dtype=object)

#### 直接读取slink.csv

In [ ]:
# slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

In [48]:
slink_df.head(2).values

array([['c13d5c79b7bd1071eeb288d809c0ab52',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt'),
        '/home/chenyi/workspace/dataset/data/data_fashionpedia/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg',
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt'),
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/images/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg')],
       ['20170510091134139',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/train/20170510091134139.txt'),
        '/home/chenyi/workspace/dataset/data/haowei34k/images/train/20170510091134139.jpg',
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/labels/train/20170510091134139.txt'),
        PosixPath('/home/chenyi/workspace/yolov5/datasets/dat

In [49]:
slink_df.head(2)

,fname,label_sp,img_sp,label_tp,img_tp
0,c13d5c79b7bd1071eeb288d809c0ab52,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/dataset/data/data_fashi...,/home/chenyi/workspace/yolov5/datasets/dataExp...,/home/chenyi/workspace/yolov5/datasets/dataExp...
1,20170510091134139,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/dataset/data/haowei34k/...,/home/chenyi/workspace/yolov5/datasets/dataExp...,/home/chenyi/workspace/yolov5/datasets/dataExp...


In [110]:
len(slink_df)

76259

##### 新的label target path.

In [ ]:
slink_df.sort_values('fname', inplace=True)

In [ ]:
img_df = df.drop_duplicates('fname').sort_values('fname')

In [ ]:
len(df), len(img_df), len(slink_df)

In [ ]:
slink_df['label_sp'] = img_df['label_tpath'].values

##### 生成label_tp, img_sp, img_tp

In [94]:
slink_df.tail(2).values

array([['20141221115344400',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/val/20141221115344400.txt'),
        '/home/chenyi/workspace/dataset/data/haowei34k/images/val/20141221115344400.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/labels/val/20141221115344400.txt',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/images/val/20141221115344400.jpg'],
       ['20140305215504279',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/val/20140305215504279.txt'),
        '/home/chenyi/workspace/dataset/data/haowei34k/images/val/20140305215504279.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/labels/val/20140305215504279.txt',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/images/val/20140305215504279.jpg']],
      dtype=object)

In [89]:
origin_folder = 'gall10k_delBox'
folder_name = 'dadetGallv4_del'

In [93]:
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: str(x).replace(f'/dataExps/{origin_folder}/images/', f'/dataExps/{folder_name}/images/'))
slink_df['label_tp'] = slink_df['label_tp'].apply(lambda x: str(x).replace(f'/dataExps/{origin_folder}/labels/', f'/dataExps/{folder_name}/labels/'))

In [ ]:
# slink_df.to_csv('./data/hw34k_level1_slink.csv', index=False)

#### 生成sym link

In [88]:
folder_name = 'dadetGallv4_del'
root = f'/home/chenyi/workspace/yolov5/datasets/dataExps'
!tree  -d -L 1 $root

/home/chenyi/workspace/yolov5/datasets/dataExps
├── dadetGallv4_del
└── gall10k_delBox

2 directories


In [95]:
folder_name = 'dadetGallv4_del'
root = f'/home/chenyi/workspace/yolov5/datasets/dataExps/{folder_name}'
!tree  -d -L 2 $root

/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [96]:
slink_df.head(2).values

array([['c13d5c79b7bd1071eeb288d809c0ab52',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt'),
        '/home/chenyi/workspace/dataset/data/data_fashionpedia/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/labels/train/c13d5c79b7bd1071eeb288d809c0ab52.txt',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/images/train/c13d5c79b7bd1071eeb288d809c0ab52.jpg'],
       ['20170510091134139',
        PosixPath('/home/chenyi/workspace/dataset/labels/labels6/labels_exp/dadetGallv4_del/labels/train/20170510091134139.txt'),
        '/home/chenyi/workspace/dataset/data/haowei34k/images/train/20170510091134139.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/labels/train/20170510091134139.txt',
        '/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/images/train/20170

In [81]:
# for p in glob.glob(str(slink_df['img_tp'][0].parents[1]/'*/*.jpg')):
#     if Path(p).is_symlink():
#         Path(p).unlink()

In [83]:
# for p in glob.glob(str(slink_df['label_tp'][0].parents[1]/'*/*.txt')):
#     if Path(p).is_symlink():
#         Path(p).unlink()

In [97]:
%%time
for p in slink_df['img_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['img_tp']), Path(row['img_sp'])), axis=1)

CPU times: user 1.64 s, sys: 974 ms, total: 2.62 s
Wall time: 2.66 s


In [98]:
%%time
for p in slink_df['label_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['label_tp']), Path(row['label_sp'])), axis=1)

CPU times: user 1.76 s, sys: 885 ms, total: 2.65 s
Wall time: 2.71 s


In [100]:
Path(slink_df['label_tp'][0]).parents[1]

PosixPath('/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del/labels')

In [102]:
len(glob.glob(str(Path(slink_df['img_tp'][0]).parents[1]/'*/*.jpg'))), len(glob.glob(str(Path(slink_df['label_tp'][0]).parents[1]/'*/*.txt')))

(76259, 76259)

In [103]:
p = str(slink_df['label_tp'][0])
!cat $p

0 0.461933 0.877930 0.074671 0.136719
0 0.528551 0.830566 0.096633 0.125977
5 0.533675 0.461426 0.341142 0.471680


#### 核查软连接

In [ ]:
import glob,cv2, os
import pandas as pd

In [ ]:
!tree  -d -L 1 /home/chenyi/workspace/yolov5/datasets/

In [105]:
# folder_name = 'dadet100kv2'
root = f'/home/chenyi/workspace/yolov5/datasets/dataExps/{folder_name}'
!tree  -d -L 2 $root

/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [106]:
def check_syslink(root, mode='images'):
    subfix = '.txt' if mode=='labels' else '.jpg'
    flag_list = [os.path.isfile(os.readlink(p)) for p in glob.glob(f'{root}/{mode}/*/*{subfix}')]
    txt_df = pd.DataFrame()
    txt_df['flag'] = flag_list
    return txt_df.value_counts('flag')

In [107]:
root

'/home/chenyi/workspace/yolov5/datasets/dataExps/dadetGallv4_del'

In [108]:
%%time
check_syslink(root, mode='images')

CPU times: user 186 ms, sys: 164 ms, total: 350 ms
Wall time: 365 ms


flag
True    76259
dtype: int64

In [109]:
%%time
check_syslink(root, mode='labels')

CPU times: user 156 ms, sys: 201 ms, total: 357 ms
Wall time: 382 ms


flag
True    76259
dtype: int64